In [4]:
# 인스타그램 쿠키 저장

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pickle

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(40)

with open("instagram_cookies.pkl", "wb") as f :
    pickle.dump(driver.get_cookies(), f)

print("쿠키 저장 완료")
driver.quit()

쿠키 저장 완료


In [5]:
# 인스타그램 키워드검색 및 댓글 크롤링

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pickle
import re

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.instagram.com/")

with open("instagram_cookies.pkl", "rb") as f :
    cookies = pickle.load(f)

for cookie in cookies :
    driver.add_cookie(cookie)

driver.refresh()
time.sleep(5)

try :
    not_now = driver.find_element(By.XPATH, '//button[text()="나중에 하기"]')
    not_now.click()
    time.sleep(2)
except :
    pass

print("정상적인 로그인 후 나중에하기 버튼 클릭 완료")

keyword = "올리브영"
driver.get(f"https://www.instagram.com/explore/tags/{keyword}/")
time.sleep(5)

base_url = "https://www.instagram.com"
post_link = set()

for _ in range(2) :
    links = driver.find_elements(By.TAG_NAME, "a")
    for link in links :
        href = link.get_attribute("href")
        if href and "/p/" in href :
            if href.startswith("/p/") :
                href = base_url + href
            post_link.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

results = []

for url in list(post_link) :
    driver.get(url)
    time.sleep(3)

    while True :
        try :
            more_btn = driver.find_element(By.XPATH, "//span[contains(text(), '더 보기')]")
            more_btn.click()
            time.sleep(1.5)
        except :
            break
            
    comments = []
    try :
        spans = driver.find_elements(By.XPATH, "//div[contains(@class, 'html-div')]//span[@dir='auto']")

        for span in spans :
            txt = span.get_attribute("textContent").strip()

            if not txt :
                continue
            if re.match(r'^\d+\s*(주|일|시간|분|초)$', txt) :
                continue
            if len(txt) <= 2 :
                continue

            comments.append(txt)
    
    except Exception as e :
        print(f"댓글 추출 실패 : {url} | 에러 : {e}")

    results.append({
        "url": url,
        "comments": comments
    })
    
driver.quit()

for r in results :
    print(r["url"])
    print(r["comments"])
    print("="*60)

정상적인 로그인 후 나중에하기 버튼 클릭 완료
https://www.instagram.com/p/DErYV6JIXPe/
['unnies_all', 'unnies_all', 'unnies_all인증됨\xa050주진짜 올영가서 왜 고민했지,,,(´･ω･`)?그냥 이거 써!! 완전 꿀템 추천해줌!😉@@살까말까 했던 거 다 있음#올리브영 #올리브영추천 #올영추천템 #뷰티 #뷰티템 #뷰스타그램', 'unnies_all', 'xiiir._.ngbb', 'xiiir._.ngbb', '@shmleh_ 너가 전에 추천해줬던거 여기있음?!', '답글 달기', '답글 1개 모두 보기', 'vussen_project', 'vussen_project', '답글 달기', 'fyjh._.06', 'fyjh._.06', '젤클렌저 쓰고있는데 요철없어지는건 모르겠던데..?피부타입은 건성인데 쫌당김있음', '답글 달기', 'haeinji.official', 'haeinji.official', '맞팔해요 ~*', '답글 달기', 'ek_.dnf', 'ek_.dnf', '여러분..저 덴티스테 꼭쓰세요..진짜레전드', '좋아요 1개', '답글 달기', 'joy.1839y', 'joy.1839y', '@mini0.315', '답글 달기', '2.s_____h', '2.s_____h', '@hipps__ong 오 대박', '답글 달기', 'ggwp.gg__', 'ggwp.gg__', '@ddd_dob 이거봐바', '답글 달기', '1월 11일', 'Meta', '블로그', '채용 정보', '도움말', 'API', '개인정보처리방침', 'Instagram Lite', 'Meta AI', 'Threads', '연락처 업로드 & 비사용자', 'Meta Verified', '한국어', '© 2025 Instagram from Meta', '탐색 탭', '메시지', '만들기', '프로필', '더 보기', 'Meta의 다른 앱', '메시지']
https://www.instagram.com/p/DQOtI8PgQJ

In [6]:
# 2차 전처리 및 엑셀파일 저장

In [13]:
import pandas as pd

rows = []
no = 1

for item in results :
    url = item["url"]
    for comment in item["comments"] :
        rows.append({
            "No": no,
            "Url": url,
            "Comment": comment.strip()
        })
        no += 1
        
df_raw = pd.DataFrame(rows)

df_clean = df_raw.copy()

df_clean = df_clean[df_clean["Comment"].str.len() > 10]

id_pattern = r'^[A-Za-z0-9_\.]+$'
df_clean = df_clean[~df_clean["Comment"].str.match(id_pattern, na=False)]

df_clean = df_clean[df_clean["Comment"].str.contains(r'[가-힣]', na=False)]

reply_view_pattern = r'^\s*답글\s*\d+\s*개\s*(?:모두\s*)?보기\s*$'
df_clean = df_clean[~df_clean["Comment"].str.match(reply_view_pattern, na=False)]

excel_path = "oliveyoung_comments_clean.xlsx"
df_clean.to_excel(excel_path, sheet_name="Comments", index=False)

print(f"엑셀파일 저장완료 : {excel_path}")

엑셀파일 저장완료 : oliveyoung_comments_clean.xlsx


In [14]:
# 형태소분석 및 엑셀파일 저장

In [17]:
import pandas as pd
from kiwipiepy import Kiwi

excel_path = "oliveyoung_comments_clean.xlsx"

df_raw = pd.read_excel(excel_path, sheet_name="Comments")
comments = df_raw["Comment"].astype(str).fillna("").tolist() # 비어있는 값 = 결측치 = NaN

kiwi = Kiwi()

tokens = []
for sent in comments :
    for token in kiwi.tokenize(sent) :
        if token.tag.startswith("N") and len(token.form) > 1 :
            tokens.append(token.form)

print(tokens)

['인증', '고민', '이거', '추천', '추천', '여기', '클렌저', '피부', '타입', '건성', '여러분', '덴티스테', '레전드', '이거봐바', '연락처', '업로드', '사용자', '수정', '올리브', '하나로', '올리브', '합격', '게시물', '저장', '친구', '공유', '공고', '취업', '자료', '소식', '스펙', '정보', '도움', '팔로우', '여러분', '저거', '슈퍼', '패스', '서류', '합격', '매장', '연락처', '업로드', '사용자', '인증', '뛰드지기', '어디', '뛰드지기', '애착', '필수', '수상', '최대', '할인', '판매', '1+1', '리무버카라', '용기', '진저슈', '에센셜', '립밤', '인증', '에뛰드', '진저슈', '세럼', '키티', '콜라', '메이크업', '요철', '짜증', '요새', '플랫', '앰프', '모공', '조음', '연락처', '업로드', '사용자', '인증', '사랑', '제품', '모두', '사랑', '혜택', '동안', '뷰티', '웰니스', '트렌드', '우리', '애정', '선정', '연말', '트렌드', '결산', '행사', '올리브', '어워즈', '국내', '글로벌', '구매', '데이터', '트렌드', '종합', '분석', '기반', '부문', '상품', '선정', '모두', '사랑', '제품', '내년', '사랑', '제품', '올리브', '어워즈', '확인', '인증', '프리미엄', '카테고리', '예전', '어워즈', '헤라블쿠', '인증', '안녕', '연락처', '업로드', '사용자', '수정', '올해', '이거', '후회', '컷팅', '젤리', '프라이머', '클렌징', '플랫', '앰프', '모공', '타이트', '효과', '덴티스테', '치약', '에르메슼', '개비쌈', '불소', '팔레트', '함정', '우리', '위시템', '컷팅', '젤리', '언니', '알고리즘', '해킹', '연락처',